In [1]:
##Binomial Tree- Call

import numpy as np
N=12
T=0.25
dt=T/N
S0=100
K=95
r=0.03
div=0.02
sigma=0.2
u = np.exp(sigma * np.sqrt(dt))
d = 1/u
p = (np.exp((r-div)*dt)-d)/ (u - d)


stock_prices = np.zeros((N, N))
call_prices = np.zeros((N, N))

stock_prices[0,0] = S0


for i in range(1, N):
    M = i + 1
    stock_prices[i, 0] = d * stock_prices[i-1, 0]
    for j in range(1, M ):
        stock_prices[i, j] = u * stock_prices[i - 1, j - 1]

expiration = stock_prices[-1,:] - K
expiration = np.where(expiration >= 0, expiration, 0)


call_prices[-1,:] =  expiration


for i in range(N - 2,-1,-1):
    for j in range(i + 1):
        call_prices[i,j] = np.exp(-r*dt) * ((1-p) * call_prices[i+1,j] + p * call_prices[i+1,j+1])


print(call_prices[0,0])

6.918695403001518


In [2]:
from scipy.stats import norm

In [3]:
##Black Scholes -Gaussian

def BlackScholes(r,S0,K,T,sigma,div):
    d1=(np.log(S0/K) + (r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2=d1-sigma*np.sqrt(T)

    price=(S0*np.exp(-div*T))*norm.cdf(d1,0,1) - K*np.exp(-r*T)*norm.cdf(d2,0,1)

    return price
    

In [4]:
BlackScholes(0.03,100,95,0.25,0.2,0.02)

7.0107551333798455

In [5]:
##American Call Binomial

import numpy as np
import matplotlib.pyplot as plt


N=12
t=0.25
dt=t/N
S0=100
K=95
r=0.03
div=0.02
sigma=0.2
u=np.exp(sigma*np.sqrt(dt))
d=1/u

p=(np.exp((r-div)*dt)-d)/(u-d)
stock_prices=np.zeros((N,N))
option_prices=np.zeros((N,N))

stock_prices[0,0]=S0

for i in range (1,N):
    M=i+1
    stock_prices[i,0]=d*stock_prices[i-1,0]
    for j in range (1,M):
        stock_prices[i,j]=u*stock_prices[i-1,j-1]

expiration=stock_prices[-1,:]-K
expiration=np.where(expiration>=0,expiration,0)


option_prices[-1,:]=expiration


for i in range(N - 2,-1,-1):
    for j in range(i+1):
        option_prices[i,j]=np.exp(-r*dt) * ((1-p) * option_prices[i+1,j] + p * option_prices[i+1,j+1])
        if stock_prices[i,j]-K>option_prices[i,j]:
            option_prices[i,j]=stock_prices[i,j]-K
        

print(option_prices[0,0])

6.918695403001518


In [6]:
##Finite Difference

def fin_diff(S,K,T,sigma,r,q,N,k):
    dt = T/N;   
    dx = sigma*np.sqrt(3*dt)
    nu = r - q - 0.5*sigma**2
    pu = 0.5*dt*((sigma/dx)**2 + nu/dx)
    pm = 1.0 - dt*(sigma/dx)**2 - r*dt  
    pd = 0.5*dt*((sigma/dx)**2 - nu/dx)
    grid = np.zeros((N+1,2*k+1))
    

    St = [S*np.exp(-k*dx)]
    for j in range(1, 2*k+1):
        St.append(St[j-1]*np.exp(dx))
    
    
    for j in range(2*k+1):
            grid[N,j] = max(0, St[j] - K)
        
    
    
    for i in range(N-1, -1, -1):
        for j in range(1, 2*k):
            grid[i,j] =pu*grid[i+1,j+1] + pm*grid[i+1,j] + pd*grid[i+1,j-1]
    
     
        grid[i,0] = grid[i,1]
        grid[i,2*k] = grid[i,2*k-1] + (St[2*k]-St[2*k-1])
    
    return grid[0,k]

In [7]:
fin_diff(100,95,0.25,0.2,0.03,0.02,8,3)

7.018645027227497